# TF-IDF

TF-IDF（Term Frequency–Inverse Document Frequency）是一种用于信息检索与文本挖掘的常用加权技术。TF-IDF可以帮助筛选出在文档中具有重要意义的词汇，常用于关键词提取、文档相似度计算和文本分类等任务。

TF-IDF 是一种统计方法，用以评估一字词对于一个文档集或一个语料库中的其中一份文档的重要程度。字词的重要性随着它在文档中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。简单的解释为，一个单词在一个文档中出现次数很多，同时在其他文档中出现此时较少，那么我们认为这个单词对该文档是非常重要的。

TF（Term Frequency，词频），某个词条在文中出现的次数。
- $词频（TF）=某个词在文档中出现的次数/文档的总词数$
IDF（Inverse Document Frequency，逆向文档频率）
- $逆文档频率（IDF）=log(语料库的文档总数/(包含该词的文档数+1))$

[NLP--词频统计和TF-IDF总结【实践】](https://blog.csdn.net/liu_673/article/details/130509959?spm=1001.2014.3001.5502)

## 优点与缺点
优点：

● 简单易实现。
● 考虑了词的重要性，减少了常见但不重要的词（如“的”、“与”等）的权重。
● 适用于自然语言处理中的多种任务，如文本分类、文本聚类、信息检索等。

缺点：

● 对于大型文档集，由于词汇量庞大，生成的 TF-IDF 矩阵往往非常稀疏，所以计算效率不高。
● 只考虑了词汇的出现频率，而忽略了词汇在文本中的顺序信息。
● 没有考虑词汇之间的语义关系。
● TF-IDF 的计算方式导致它的效果依赖于足够大的文档集，数据量不足可能导致效果不佳。

## 应用场景

● 关键词提取
● 文档相似度计算
● 信息检索
● 文本分类

In [16]:
import os
import math
import pandas as pd

# 读取文件路径
file_path = os.path.join("./data", "corpus.txt")

# 从文件中读取语料库
with open(file_path, encoding="utf-8") as f:
    corpus = f.readlines()

# 构建词汇表
vocab = sorted(set(" ".join(corpus).split()))
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
vocab_size = len(vocab)

# 计算文档频率
doc_freq = {word: 0 for word in vocab}
for sentence in corpus:
    unique_words = set(sentence.split())
    for word in unique_words:
        if word in doc_freq:
            doc_freq[word] += 1

# 计算逆文档频率(IDF)
num_docs = len(corpus)
# idf = {word: math.log(num_docs / (1 + df)) + 1  for word, df in doc_freq.items()}
idf = {word: math.log((1 + num_docs) / (1 + df)) + 1 for word, df in doc_freq.items()}
# 平滑处理：在计算 IDF 时，分母加 1，以避免除零错误，并确保 IDF 值始终大于 0

# 将文本转换为TF-IDF表示
def text_to_tfidf(text, idf, vocab_size, word_to_idx):
    tfidf = [0] * vocab_size
    words = text.split()

    # 计算词频(TF)
    word_count = {}
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1

    # 计算TF-IDF
    for word, count in word_count.items():
        if word in idf:
            idx = word_to_idx[word]
            tfidf[idx] = (count / len(words)) * idf[word]

    return tfidf

# 将语料库中的每句话转换为TF-IDF表示
tfidf_corpus = [text_to_tfidf(sentence, idf, vocab_size, word_to_idx) for sentence in corpus]

# 打印TF-IDF表示结果
# for sentence, tfidf in zip(corpus, tfidf_corpus):
#     print(f"句子: '{sentence.strip()}'")
#     print(f"词汇: {vocab}")
#     print(f"TF-IDF表示: {tfidf}")

df_tfidf = pd.DataFrame(tfidf_corpus, columns=vocab)
df_tfidf

# 如果想将结果保存到CSV文件中，可以使用以下代码
# df_tfidf.to_csv('tfidf_results.csv', index=False)

,John,Mary,also,football,games.,likes,"moives,",to,too.,watch
0,0.125000,0.175683,0.000000,0.000000,0.000000,0.250000,0.175683,0.125000,0.175683,0.125000
1,0.142857,0.000000,0.200781,0.200781,0.200781,0.142857,0.000000,0.142857,0.000000,0.142857


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 使用TfidfVectorizer初始化一个TF-IDF向量化器
vectorizer = TfidfVectorizer(stop_words=None, lowercase=False)

# 学习语料库并进行向量化
X = vectorizer.fit_transform(corpus)

# 获取词汇表
vocab = vectorizer.get_feature_names_out()

# 打印每条记录的TF-IDF值
# for sentence, tfidf in zip(corpus, X.toarray()):
#     print(f"句子: '{sentence.strip()}'")
#     print(f"{vocab}")
#     print(f"TF-IDF表示: \n{tfidf}\n")
df = pd.DataFrame(X.toarray(), columns=vocab)
df

,John,Mary,also,football,games,likes,moives,to,too,watch
0,0.278143,0.39092,0.000000,0.000000,0.000000,0.556286,0.39092,0.278143,0.39092,0.278143
1,0.317404,0.00000,0.446101,0.446101,0.446101,0.317404,0.00000,0.317404,0.00000,0.317404
